# Latihan-9a Simple 1D Model

Latihan ini untuk pemodelan half space atau disebut juga blocky model, atau interface model yang terdiri dari 2-3 block layers digunakan untuk menganalisis respon amplitudo terhadap interface antara top dan bottom layers
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-9 Simple 1D model.ipynb"_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Program setup dan display sama seperti latihan sebelumnya. Gunakan hasil dari Latihan-3 data _logs_blocked_ untuk latihan selanjutnya
<br> 
        
        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        The bruges library's credits go to contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from matplotlib import cm, pyplot
from scipy.optimize import curve_fit
from scipy.signal import hilbert
from f07reflect import fullzoep, ei_norm
from f07gassmann import fluidsub
from brugeslibrary import vels, stiffsand, softsand, hertzmindlin, contactcement, vrh
mpl.style.use('default') # default, seaborn

__Tahap-2__ 
<br> Mendapatkan nilai rata-rata untuk masing-masing interval<br>
nilai top pick untuk 5(lima) layers dari Latihan-3 sebelumnya adalah sbb:

In [ ]:
adi2=pd.read_csv('adi02block.csv')
z=adi2.Depth

top_plot=1650
pick1=1680
pick2=1720
pick3=1740
pick4=1770
base_plot=1800

menghitung interval property arrays dari 5-layers, dan visualisasi asumsi litofasies dan properti elastiknya. Menggunakan file lain di luar halaman ini yang bernama _09visual_adi02blk.py_ 

In [ ]:
exec(open('f09visual_adi02blk.py').read())

Checkshot data memberikan hubungan Time-Depth sbb __TWT(ms) = 0.6395 * Depth(m) + 383.52__. Sehingga semua properti dapat dikonversi dalam domain waktu

In [ ]:
adi2.head(5);

__Tahap-3__ 
<br> Membuat model dari nilai rata-rata untuk masing-masing interval<br><br>
Copy array dari hasil Latihan-8 untuk simplifikasi sbb:

        thickness =[30, 40, 20, 30, 30]
        layer1_elog =[3.3, 1.85, 2.42]
        layer2_elog =[3.28, 1.77, 2.44]
        layer3_elog =[3.08, 1.72, 2.25]
        layer4_elog =[3.41, 1.96, 2.32]
        layer5_elog =[3.48, 1.85, 2.48]
        vplyr=[3.3, 3.28, 3.08, 3.41, 3.48]
        vslyr=[1.85, 1.77, 1.72, 1.96, 1.85]
        rholyr=[2.42, 2.44, 2.25, 2.32, 2.48]
        vshlyr=[0.58, 0.71, 0.23, 0.34, 0.76]
        phitlyr=[0.12, 0.12, 0.21, 0.16, 0.11]
        swtlyr=[0.95, 0.98, 0.75, 0.84, 1.0]

In [ ]:
#input : copy yang diperlukan saja
thickness =[30, 40, 20, 30, 30]
vplyr_i=[3.3, 3.28, 3.08, 3.41, 3.48]
vslyr_i=[1.85, 1.77, 1.72, 1.96, 1.85]
rholyr_i=[2.42, 2.44, 2.25, 2.32, 2.48]

#simplifikasi blook layers di atas, agar prosesnya mudah diubah dan dibahas dalam kelas
def make_log(log, thickness):
    out = []
    for thick, x in zip(thickness, log):
        out += thick * [x]
    time = np.arange(0, sum(thickness))
    return np.array(out), time
vplyr, time = make_log(vplyr_i, thickness)
vslyr, _ = make_log(vslyr_i, thickness)
rholyr, _ = make_log(rholyr_i, thickness)
vpvs = vplyr/vslyr
# acoustic impedance
ip = vplyr * rholyr
ip_lower = ip[1:]
ip_upper = ip[:-1]
rc = (ip_lower - ip_upper) / (ip_lower + ip_upper)
"""
default wavelet yang digunakan adalah Ricker central frequency 20Hz, 200samples
eksplorasi pilihan rotasi wavelet dengan mengubah ph=0 menjadi rotasi yang diinginkan misalnya ph=-90deg
"""
f, l, dt, ph = 20, 200, 2, 0
t = np.arange(-l/2, l/2, dt)/1000
zerophase = (1.0 - 2.0*(np.pi**2)*(f**2)*(t**2)) * np.exp(-(np.pi**2)*(f**2)*(t**2))
h = hilbert (zerophase)
theta = ph*np.pi/180
wavelet = np.cos(theta)*h.real-np.sin(theta)*h.imag
# convolution
syn = np.convolve(rc, wavelet, mode='same')
print("Selalu QC no of samples berikut:", len(syn))

In [ ]:
def rcshuey(vp1, vs1, rho1, vp2, vs2, rho2, theta):
    a = np.radians(theta)
    dvp = vp2-vp1
    dvs = vs2-vs1
    drho = rho2-rho1
    vp  = np.mean([vp1,vp2])
    vs  = np.mean([vs1,vs2])
    rho = np.mean([rho1,rho2])
    I = 0.5*(dvp/vp + drho/rho)
    G = 0.5*(dvp/vp) - 2*(vs**2/vp**2)*(drho/rho+2*(dvs/vs))
    F = 0.5*(dvp/vp)
    R = I + G*np.sin(a)**2 + F*(np.tan(a)**2-np.sin(a)**2)
    return R

In [ ]:
vplo = vplyr[1:]; vslo = vslyr[1:];rholo = rholyr[1:]
vpup = vplyr[:-1]; vsup = vslyr[:-1]; rhoup = rholyr[:-1]
angles = 40
traces = np.linspace(0, angles, angles)
rcavo=[]; synavo = []
for i in range(angles):
    rcavo = rcshuey(vpup, vsup, rhoup, vplo, vslo, rholo, theta=i)
    syn = np.convolve(wavelet, rcavo, mode='same') 
    synavo.append(syn)

print("Selalu QC no of samples berikut:", len(synavo[0]))

In [ ]:
#plot
fig, axs = plt.subplots(ncols=6, figsize=(10, 5), sharey=True)
ax0 = axs[0]; ax0.plot(vplyr, time, 'k'); ax0.invert_yaxis()
ax0.set_xlabel('Vp',color='k', fontsize=12) 
ax1 = axs[1]; ax1.plot(vslyr, time, 'k')
ax1.set_xlabel('Vs',color='k', fontsize=12) 
ax2 = axs[2]; ax2.plot(rholyr, time, 'k')
ax2.set_xlabel('Rho',color='k', fontsize=12) 
ax3 = axs[3]; ax3.plot(synavo[5], time[:-1], 'k')
ax3.fill_betweenx(time[:-1], 0, synavo[5], 
                  where=(synavo[5]>0),color='b', alpha=0.6)
ax3.fill_betweenx(time[:-1], 0, synavo[5], 
                  where=(synavo[5]<0),color='r', alpha=0.6)
ax3.set_xlabel('Near=5deg',color='k', fontsize=12) 
ax4 = axs[4]; ax4.plot(synavo[15],time[:-1],'k')
ax4.plot(synavo[5], time[:-1], 'k', lw=1.5, linestyle=":")
ax4.fill_betweenx(time[:-1], 0, synavo[15], 
                  where=(synavo[15]>0),color='b', alpha=0.6)
ax4.fill_betweenx(time[:-1], 0, synavo[15], 
                  where=(synavo[15]<0),color='r', alpha=0.6)
ax4.set_xlabel('Mid=15deg',color='k', fontsize=12) 
ax5 = axs[5]; ax5.plot(synavo[35],time[:-1],'k')
ax5.plot(synavo[5], time[:-1], 'k', lw=1.5, linestyle=":")
ax5.fill_betweenx(time[:-1], 0, synavo[35], 
                  where=(synavo[35]>0),color='b', alpha=0.6)
ax5.fill_betweenx(time[:-1], 0, synavo[35], 
                  where=(synavo[35]<0),color='r', alpha=0.6)
ax5.set_xlabel('Far=35deg',color='k', fontsize=12) 
top=thickness[0]+thickness[1]
base=thickness[0]+thickness[1]+thickness[2]
for aa in axs:
    aa.xaxis.set_label_position('top')
    aa.axhline(y=top, linewidth=1, color='r')
    aa.axhline(y=base, linewidth=1, color='b')
for bb in [ax3, ax4, ax5]:
    bb.set_xlim(-0.25,0.25)
plt.show()

__Tahap-4__ 
<br> Variasi porositas, saturasi dan Vsh

### Porosity, Saturation and Vsh

In [ ]:
vshlyr_i=[0.58, 0.71, 0.23, 0.34, 0.76]
phitlyr_i=[0.12, 0.12, 0.21, 0.16, 0.11]
swtlyr_i=[0.95, 0.98, 0.75, 0.84, 1.0]
vshlyr, _ = make_log(vshlyr_i, thickness)
phitlyr, _ = make_log(phitlyr_i, thickness)
swtlyr, _ = make_log(swtlyr_i, thickness)
#plot
f, axes = plt.subplots(ncols=3, figsize=(6, 5), sharey=True)
ax0 = axes[0]; ax0.plot(vplyr, time, 'k'); ax0.invert_yaxis()
ax0.set_xlabel('Vp',color='k', fontsize=12) 
ax1 = axes[1]; ax1.plot(vshlyr, time, 'k')
ax1.set_xlabel('Vsh',color='k', fontsize=12) 
ax2 = axes[2]; ax2.plot(phitlyr, time, 'k')
ax2.set_xlabel('Phit',color='k', fontsize=12)
top=thickness[0]+thickness[1]
base=thickness[0]+thickness[1]+thickness[2]
for aa in axes:
    aa.xaxis.set_label_position('top')
    aa.axhline(y=top, linewidth=1, color='r')
    aa.axhline(y=base, linewidth=1, color='b')
plt.show()

### Velocity-Property Relations

pemodelan variasi reservoir properties dilandasi dengan model fisika batuan yang sesuai dengan kondisi data. Gunakan pilihan yang ada seperti yang dijelaskan dalam kursus sebelumnya, e.g.: Fundamental Rock Physics dan Fundamental QI

            Bruges functions:
            hertzmindlin(K0, G0, phic=0.4, Cn=8.6, sigma=10, f=1)
            stiffsand(K0, G0, phi, phic=0.4, Cn=8.6, sigma=10 Peff, f=1)
            softsand(K0, G0, phi, phic=0.4, Cn=8.6, sigma=10 Peff, f=1)
            vels(K_DRY,G_DRY,K0=Kma,D0=Rhoma,Kf=Kfl,Df=rhofluid,phi)
            vrh(volumes,k,mu)

In [ ]:
rho_qz=2.65;  k_qz=37;  mu_qz=44    # mineral properties, quartz (i.e., sands)
rho_sh=2.81;  k_sh=15;  mu_sh=5     # mineral properties, clay (i.e., shales)
rho_b=1.09;   k_b=2.8               # fluid properties, brine
rho_o=0.78;   k_o=0.94              # fluid properties, oil
rho_g=0.25;   k_g=0.06              # fluid properties, gas

In [ ]:
""" input rock physics models"""
vsh=vshlyr
vsd=1-vsh
phi=phitlyr
sw=swtlyr
kvoigt=(vsh*k_sh) + (vsd*k_qz)
kreuss=1/(vsh/k_sh + vsd/k_qz)
kvrh=0.5*(kvoigt+kreuss)
K0 = kvrh
muvoigt=(vsh*mu_sh) + (vsd*mu_qz)
mureuss=1/(vsh/mu_sh + vsd/mu_qz)
muvrh=0.5*(muvoigt+mureuss)
G0 = muvrh
D0 = vsh*(rho_sh)+(1-vsh)*rho_qz 
Df = sw*rho_b+(1-sw)*rho_o
Kf = 1/((1-sw)/k_o + sw/k_b)
M0=K0+(4/3)*G0
Vp0=np.sqrt(M0/D0)
Vpf=np.sqrt(Kf/Df)

### Model Rock Physics

Beberapa model variasi contact model berdasarkan Hertz-mindlin dapat digunakan. Seperti model _stiff sand, soft sand, contact cement_ dan _critical porosity_ atau model lain. Model rock physics ini dibahas dalam kelas Fundamental Rock Physics dan Fundamental QI

<br> Model variasi velocity dan porosity yang digunakan di bawah misalnya Raymer, et.al (1980) juga dapat dieksplorasi untuk fitting data trend. Hubungan Raymer diberikan sebagai berikut:
<br>
   
<font size=4>$V = (1-\phi)^2.V_0 + \phi.V_{fl}$</font>, untuk $\phi$ < 37%
<br>

<font size=5>$\frac{1}{\rho.V^2} = \frac{\phi}{\rho_{fl}.V_{fl}^2} + \frac{1-\phi}{\rho_{ma}.V_{ma}^2} $</font>, untuk $\phi$ > 47%
<br><br>
untuk intermediate porosities:
<br>

<font size=5>$\frac{1}{V} = \frac{(0.47-\phi)}{0.10}.\frac{1}{V_{37}} + \frac{(\phi-0.37)}{0.10}.\frac{1}{V_{47}}$ </font>
<br><br>

dimana $V_{37}$ dihitung dari hubungan low porosity pada $\phi$=0.37, dan $V_{47}$ dihitung dari hubungan high porosity pada $\phi$=0.47


    Catatan: model rock physics Raymer, hanya contoh penjelasan ilustrasi di bawah. 
    Gunakan model rock physics lain yang sesuai dengan tren dan variasi data yang ada

------------------------------------------------------------------------

## Pemodelan Perubahan Porositas

In [ ]:
# velocity - Porosity
"""soft sand model"""
soft=softsand(K0, G0, phi, phic=0.3, Cn=5, sigma=4, f=1)
Kdrysoft=soft[0]; Gdrysoft=soft[1]
velsoft=vels(Kdrysoft,Gdrysoft,K0,D0,Kf,Df,phi)
vpsoft=0.001*velsoft[0];vssoft=0.001*velsoft[1]
rhosoft=velsoft[2];Ksoft=velsoft[3]
# print(len(vplyr),len(vpsoft))
""" Raymer et.al Vp-porosity model for Phi <0.37 """
vpray=(((1-phitlyr)**2)*Vp0)+(phitlyr*Vpf)

Variasi model pengurangan velocity karena meningkatnya porositas dan clay content juga telah dimodelkan oleh Han, menggunakan sampel shaley sand, pada kondisi variasi pressure secara umum direprentasikan dalam hubungan linier (Mavko et.al., The Rock Physics Handbook, 2nd ed.):
   
<font size=4>$V = a - b\phi - c V_{cl}$</font> <br>

|Clean Sand|||||
|:----|----|----|---:|:--:|
|Water Sat|Vp_a|Vp_b|Vs_a|Vs_b|
|40MPa|6.08|8.06|4.06|6.28|

|Shaley Sand|||||||
|:----|----|----|----|----|---:|:--:|
|Water Sat|Vp_a|Vp_b|Vp_c|Vs_a|Vs_b|Vs_c|
|:----|----|----|----|----|---:|:--:|
|40Mpa|5.59|6.93|2.18|3.52|4.91|1.89|
|30Mpa|5.55|6.96|2.18|3.47|4.84|1.87|
|20Mpa|5.49|6.94|2.17|3.39|4.73|1.81|
|10MPa|5.39|7.08|2.13|3.39|4.73|1.74|
|5MPa |5.26|7.08|2.02|3.29|4.77|1.64|    

|Dry  |Vp_a|Vp_b|Vp_c|Vp_a|Vp_b|Vp_c|
|:----|----|----|----|----|---:|:--:|
|40MPa|5.41|6.35|2.87|3.57|4.57|1.83|

In [ ]:
# -------fitting (not good! as per July 1, 2021)----------
def func(X, a, b, c):
    x,y = X
    return a - b*x - c*y
x = phitlyr
y = vshlyr
a, b, c = 5.49, 6.94, 2.17
z = func((x,y), a, b, c)*1 + np.random.random(150)
# initial guesses for a,b,c:
g0 = 5.49, 6.94, 2.17
print (curve_fit(func, (x,y), z, g0))

In [ ]:
# velocity - Clay
"""Han's model"""
vshcha=0.52 # fit to data. Original vsh is 0.23
Vp40Mpa=5.59-6.93*phi-2.18*vshcha; Vp30Mpa=5.55-6.96*phi-2.18*vshcha
Vp20Mpa=5.49-6.94*phi-2.17*vshcha; Vp10Mpa=5.39-7.08*phi-2.13*vshcha
Vp5Mpa =5.26-7.08*phi-2.02*vshcha; Vpfit=5.43-5.10*phi-1.89*vshcha
f, axs=plt.subplots(ncols=2, figsize=(9,4))
ax1=axs[0]
ax1.scatter(phitlyr, vplyr, s=40, facecolors='gray', edgecolors='k', label="data")
ax1.plot(phitlyr,Vp40Mpa, 'r', lw=1, label="40Mpa", alpha=0.8)
ax1.plot(phitlyr,Vp30Mpa, 'k', lw=0.5, label="30Mpa", alpha=0.6)
ax1.plot(phitlyr,Vp20Mpa, 'k', lw=0.5, label="20Mpa", alpha=0.4)
ax1.plot(phitlyr,Vp10Mpa, 'k', lw=0.5, label="10Mpa", alpha=0.2)
ax1.plot(phitlyr,Vp5Mpa, 'k', lw=0.5, label="5Mpa", alpha=0.1)
Vsh40=5.59-6.93*phi-2.18*0.5; Vsh50=5.59-6.93*phi-2.18*0.55 
Vsh60=5.59-6.93*phi-2.18*0.6; Vsh70=5.59-6.93*phi-2.18*0.65
ax2=axs[1]
ax2.scatter(phitlyr, vplyr, s=40, facecolors='gray', edgecolors='k', label="data")
ax2.plot(phitlyr,Vsh40, 'k', lw=1, label="Vsh=0.5", alpha=0.1)
ax2.plot(phitlyr,Vsh50, 'k', lw=1, label="Vsh=0.55", alpha=0.3)
ax2.plot(phitlyr,Vsh60, 'k', lw=1, label="Vsh=0.6", alpha=0.5)
ax2.plot(phitlyr,Vsh70, 'k', lw=1, label="Vsh=0.65", alpha=0.7)
for aa in axs:
    aa.set_xlabel("phit", size=14); aa.set_ylabel("vp", size=14)
    aa.set_ylim([2.5,4])
    aa.grid(linestyle=":", color="gray", alpha=0.4)
    aa.legend(loc="lower left", frameon=False, fontsize=8)
plt.show()

### Han's Fitting model menggunakan hubungan high pressure 40MPa
<br>
$Vp=5.59-6.93*phi-2.18*Vsh$, dengan variasi Vsh= 0.5, 0.55, 0.60, 0.65
<br>
$Vs=3.52-4.91*phi-1.89*Vsh$, variasi Vs tidak dimodelkan, diasumsikan konsisten dengan Vp
<br>
Rho mengikuti perubahan Vp, linear atau Gardner's based relation
<br><br>
hubungan di atas juga menunjukkan retardasi porositas akibat meningkatnya Vsh ke arah kiri bawah. Hubungan Vsh dan Phit pada data Han berbanding terbalik dan dapat juga di modelkan untuk data ini
<br><br>

        aplikasi: jika layer-3 channel bervariasi porositasnya menjadi mengecil dan berubah menjadi 
        stiffening shaley sand dengan Vsh. Maka Vp akan berubah menjadi stiffening Vp_new dan Vs_new 
        dengan faktor di atas, Rho diasumsikan mengikuti Gardner atau linier.

In [ ]:
def han(x, a, b):
    return a*x + b
fit,_=curve_fit(han,phitlyr,vshlyr)
print(fit)

In [ ]:
plt.subplots(figsize=(4,4))
plt.scatter(phitlyr, vshlyr, s=40, facecolors='gray', edgecolors='k', label="data")
plt.plot(phitlyr, (fit[0]*phitlyr)+fit[1], 'r', linestyle="--", lw=2)
plt.xlabel("phit", size=14); plt.ylabel("vsh", size=14)
plt.ylim([0.1,0.8]); plt.xlim([0.05,0.25])
plt.grid(linestyle=":", color="gray", alpha=0.4)
plt.legend(loc="lower left", frameon=False, fontsize=8)
plt.show()

In [ ]:
#--------------Replace Porosity di Layer-3 by 10%-------------
phit_i= 0.21
phit10less=0.9*0.21
vsh10less=fit[0]*phit10less + fit[1]
print(phit10less, round(vsh10less,3))  # hasilnya ==> phit=0.189 dan vsh =0.275
vp10less=5.59-6.93*phit10less-2.18*vsh10less
vs10less=3.52-4.91*phit10less-1.89*vsh10less
rho10less=1.63*vp10less**0.3
print(round(vp10less,3), round(vs10less,3), round(rho10less,3))  # hasilnya ==> vp=3.682, vs =2.073, rho=2.41

#----------new set of layer-3, menjadi sbb-----------
vshlyr10less=[0.58, 0.71, 0.275, 0.34, 0.76] # replace 0.23 menjadi 0.275
phitlyr10less=[0.12, 0.12, 0.189, 0.16, 0.11] # replace 0.21 menjadi 0.189 (menurun 10%)
vplyr10less=[3.3, 3.28, 3.682, 3.41, 3.48] # replace 3.08 menjadi 3.682 dengan model Han 40MPa
vslyr10less=[1.85, 1.77, 2.073, 1.96, 1.85] # replace 1.72 menjadi 2.073 dengan model Han 40MPa
rholyr10less=[2.42, 2.44, 2.41, 2.32, 2.48] # replace 2.25 menjadi 2.41 dengan asumsi Gardner-Sd rho=1.63Vp**0.30
vplyr10ph, time = make_log(vplyr10less, thickness)
vslyr10ph, _ = make_log(vslyr10less, thickness)
rholyr10ph, _ = make_log(rholyr10less, thickness)
vshlyr10ph, _ = make_log(vshlyr10less, thickness)
phitlyr10ph, _ = make_log(phitlyr10less, thickness)
vplo10ph = vplyr10ph[1:]; vslo10ph = vslyr10ph[1:]; rholo10ph = rholyr10ph[1:]
vpup10ph = vplyr10ph[:-1]; vsup10ph = vslyr10ph[:-1]; rhoup10ph = rholyr10ph[:-1]
angles = 40
traces = np.linspace(0, angles, angles)
rcavo10ph=[]; synavo10ph = []
for i in range(angles):
    rcavo10ph = rcshuey(vpup10ph, vsup10ph, rhoup10ph, vplo10ph, vslo10ph, rholo10ph, theta=i)
    syn10ph = np.convolve(wavelet, rcavo10ph, mode='same') 
    synavo10ph.append(syn10ph)
print("Selalu QC no of samples berikut :", len(synavo10ph[0]))

In [ ]:
#-------------visualisasi canvas-------------------------------
fig, axs2 = plt.subplots(ncols=6, figsize=(10, 5), sharey=True)        # frame dari plot, note=6 kolom, size=10x5

ax0 = axs2[0]; ax0.plot(vplyr, time, 'k', lw=2, zorder=5);             # plot kolom-1
ax0.plot(vplyr10ph, time, 'r', lw=1, zorder=1); 
ax0.fill_betweenx(time, vplyr10ph, vplyr, color='r', alpha=0.2)
ax0.invert_yaxis(); ax0.set_xlabel('Vp',color='k', fontsize=12)

ax1 = axs2[1]; ax1.plot(vslyr, time, 'k', lw=2, zorder=5);             # plot kolom-2 
ax1.plot(vslyr10ph, time, 'r', lw=1, zorder=1)
ax1.fill_betweenx(time, vslyr10ph, vslyr, color='r', alpha=0.2)
ax1.set_xlabel('Vs',color='k', fontsize=12)

ax2 = axs2[2]; ax2.plot(rholyr, time, 'k', lw=2, zorder=5);            # plot kolom-3
ax2.plot(rholyr10ph, time, 'r', lw=1, zorder=1)
ax2.fill_betweenx(time, rholyr10ph, rholyr, color='r', alpha=0.2)
ax2.set_xlabel('Rho',color='k', fontsize=12) 

ax3 = axs2[3]; ax3.plot(synavo10ph[5], time[:-1], 'k')              # plot kolom-4
ax3.plot(synavo[5], time[:-1], 'k', lw=1.5, linestyle="--")
ax3.fill_betweenx(time[:-1], 0, synavo10ph[5], 
                  where=(synavo10ph[5]>0),color='b', alpha=0.6)
ax3.fill_betweenx(time[:-1], 0, synavo10ph[5], 
                  where=(synavo10ph[5]<0),color='r', alpha=0.6)
ax3.set_xlabel('Near=5deg',color='k', fontsize=12) 

ax4 = axs2[4]; ax4.plot(synavo10ph[15],time[:-1],'k')              # plot kolom-5
ax4.plot(synavo[15], time[:-1], 'k', lw=1.5, linestyle="--")
ax4.fill_betweenx(time[:-1], 0, synavo10ph[15], 
                  where=(synavo10ph[15]>0),color='b', alpha=0.6)
ax4.fill_betweenx(time[:-1], 0, synavo10ph[15], 
                  where=(synavo10ph[15]<0),color='r', alpha=0.6)
ax4.set_xlabel('Mid=15deg',color='k', fontsize=12) 

ax5 = axs2[5]; ax5.plot(synavo10ph[35],time[:-1],'k')                 # plot kolom-6
ax5.plot(synavo[35], time[:-1], 'k', lw=1.5, linestyle="--")
ax5.fill_betweenx(time[:-1], 0, synavo10ph[35], 
                  where=(synavo10ph[35]>0),color='b', alpha=0.6)
ax5.fill_betweenx(time[:-1], 0, synavo10ph[35], 
                  where=(synavo10ph[35]<0),color='r', alpha=0.6)
ax5.set_xlabel('Far=35deg',color='k', fontsize=12) 
top=thickness[0]+thickness[1]
base=thickness[0]+thickness[1]+thickness[2]
for aa in axs2:
    aa.xaxis.set_label_position('top')
    aa.axhline(y=top, linewidth=1, color='r')
    aa.axhline(y=base, linewidth=1, color='b')
for bb in [ax3, ax4, ax5]:
    bb.set_xlim(-0.25,0.25)

plt.show()

In [ ]:
#plot new por
f, axes = plt.subplots(ncols=3, figsize=(6, 5), sharey=True)

ax0 = axes[0]; ax0.invert_yaxis()
ax0.plot(vplyr, time, 'k', zorder=1); ax0.plot(vplyr10ph, time, 'r', zorder=5)
ax0.fill_betweenx(time, vplyr10ph, vplyr, color='r', alpha=0.2)
ax0.set_xlabel('Vp',color='k', fontsize=12)

ax1 = axes[1]; ax1.plot(vshlyr, time, 'k', zorder=1)
ax1.plot(vshlyr10ph, time, 'r', zorder=1)
ax1.fill_betweenx(time, vshlyr10ph, vshlyr, color='r', alpha=0.2)
ax1.set_xlabel('Vsh',color='k', fontsize=12)

ax2 = axes[2]; ax2.plot(phitlyr, time, 'k', zorder=1)
ax2.plot(phitlyr10ph, time, 'r', zorder=1)
ax2.set_xlabel('Phit',color='k', fontsize=12)
ax2.fill_betweenx(time, phitlyr10ph, phitlyr, color='r', alpha=0.2)

top=thickness[0]+thickness[1]
base=thickness[0]+thickness[1]+thickness[2]
for aa in axes:
    aa.xaxis.set_label_position('top')
    aa.axhline(y=top, linewidth=1, color='r')
    aa.axhline(y=base, linewidth=1, color='b')
plt.show()

### Vp Porosity Raymer dan Soft Sand trend

In [ ]:
Vma=Vp0; Vf=Vpf
def fit(x,a,b):
    return a*x+b
raw,_ = curve_fit(fit, phitlyr, vplyr)
print("raw:", raw)
sof,_ = curve_fit(fit, phitlyr, vpsoft)
print("soft:",sof)

In [ ]:
plt.subplots(figsize=(4,4))
plt.scatter(phitlyr, vplyr, s=40, facecolors='k', edgecolors='k', label="blocked data")
plt.scatter(phitlyr, vpsoft, s=40, facecolors='none', edgecolors='g', label="Soft Sand")
plt.scatter(phitlyr, vpray, s=40, facecolors='none', edgecolors='b', label="Raymer")
plt.plot(phitlyr,(raw[1]+(raw[0]*phitlyr)), 'k', linestyle=":")
plt.plot(phitlyr,(sof[1]+(sof[0]*phitlyr)), 'g', linestyle=":")
plt.xlabel("phit", size=14); plt.ylabel("vp", size=14)
plt.ylim([2.0,4])
plt.grid(linestyle=":", color="gray", alpha=0.4)
plt.legend(loc="upper left", frameon=False, fontsize=6)
plt.show()

__Tahap-5__ 
<br> Variasi saturasi

## Pemodelan Perubahan Saturasi

In [ ]:
# -------layer-3 cha vp, vs, rho
vpcha=3.08; vscha=1.72; rhocha=2.25
phitcha=0.21; vshcha=0.23; swtcha=0.75

#-----------FRM dengan 80% oil sat-----------------
sofrm=0.8 # ubah untuk saturasi lainnya
rhofl_in = swtcha*rho_b+(1-swtcha)*rho_o
rhofl_out = sofrm*rho_o+(1-sofrm)*rho_b
kfl_in = 1/((1-swtcha)/k_o + swtcha/k_b)
kma_in = (vshcha*k_sh) + ((1-vshcha)*k_qz)
kfl_out = 1/((1-sofrm)/k_b + sofrm/k_o)
frmo=fluidsub(vpcha, vscha, rhocha, phitcha, rhofl_in, rhofl_out, kma_in, kfl_in, kfl_out)
pctVpo=((frmo[0]-vpcha)/(vpcha))*100
pctVso=((frmo[1]-vscha)/(vscha))*100
pctRhoo=((frmo[2]-rhocha)/(rhocha))*100

print("vp 80% oil:",round((frmo[0]),3))
print("vs 80% oil:",round((frmo[1]),3))
print("rho 80% oil:",round((frmo[2]),3))
print("-------------")
print("% change vp:",round(pctVpo,2))
print("% change vs:",round(pctVso,2))
print("% change rho:",round(pctRhoo,2))

In [ ]:
#input : copy yang diperlukan saja
thickness_adi =[30, 40, 20, 30, 30] # thickness harus sama dengan initial untuk bisa dibandingkan
vplyr_adi=[3.3, 3.28, 1.959, 3.41, 3.48] #replaced vpoil=1.959 dg angka di atas
vslyr_adi=[1.85, 1.77, 1.734, 1.96, 1.85] # replaced vs=1.734
rholyr_adi=[2.42, 2.44, 2.214, 2.32, 2.48] #replaced rho=2.214

def makenewlog(newlog, thickness_adi):
    out = []
    for thick, x in zip(thickness_adi, newlog):
        out += thick * [x]
    timeadi = np.arange(0, sum(thickness_adi))
    return np.array(out), timeadi
vplyradi, timeadi = makenewlog(vplyr_adi, thickness_adi)
vslyradi, _ = makenewlog(vslyr_adi, thickness_adi)
rholyradi, _ = makenewlog(rholyr_adi, thickness_adi)

# AVO reflectivity
vploadi = vplyradi[1:]; vsloadi = vslyradi[1:]; rholoadi = rholyradi[1:]
vpupadi = vplyradi[:-1]; vsupadi = vslyradi[:-1]; rhoupadi = rholyradi[:-1]
anglesadi = 40
tracesadi = np.linspace(0, anglesadi, anglesadi)
rcavoadi=[]; synavoadi = []
for i in range(anglesadi):
    rcavoadi = rcshuey(vpupadi, vsupadi, rhoupadi, vploadi, vsloadi, rholoadi, theta=i)
    synadi = np.convolve(wavelet, rcavoadi, mode='same') 
    synavoadi.append(synadi)

In [ ]:
print("-----------warning------------")
print("QC x-axis initial:", len(synavo[0]))
print("QC x-axis new:", len(synavoadi[0]))
print("QC time axis initial:", len(time))
print("QC time axis new:", len(timeadi))
print("----jika tidak konsisten jumlahnya, maka ulangi salah satu proses di atas-----")

In [ ]:
#-------------visualisasi canvas-------------------------------
fig, axs2 = plt.subplots(ncols=6, figsize=(10, 5), sharey=True)        # frame dari plot, note=6 kolom, size=10x5

ax0 = axs2[0]; ax0.plot(vplyr, time, 'k', lw=2, zorder=5);             # plot kolom-1
ax0.plot(vplyradi, timeadi, 'r', lw=1, zorder=1); 
ax0.fill_betweenx(time, vplyradi, vplyr, color='r', alpha=0.2)
ax0.invert_yaxis(); ax0.set_xlabel('Vp',color='k', fontsize=12)

ax1 = axs2[1]; ax1.plot(vslyr, time, 'k', lw=2, zorder=5);             # plot kolom-2 
ax1.plot(vslyradi, timeadi, 'r', lw=1, zorder=1)
ax1.fill_betweenx(time, vslyradi, vslyr, color='r', alpha=0.2)
ax1.set_xlabel('Vs',color='k', fontsize=12)

ax2 = axs2[2]; ax2.plot(rholyr, time, 'k', lw=2, zorder=5);            # plot kolom-3
ax2.plot(rholyradi, timeadi, 'r', lw=1, zorder=1)
ax2.fill_betweenx(time, rholyradi, rholyr, color='r', alpha=0.2)
ax2.set_xlabel('Rho',color='k', fontsize=12) 

ax3 = axs2[3]; ax3.plot(synavoadi[5], timeadi[:-1], 'k')              # plot kolom-4
ax3.plot(synavo[5], time[:-1], 'k', lw=1.5, linestyle="--")
ax3.fill_betweenx(time[:-1], 0, synavoadi[5], 
                  where=(synavoadi[5]>0),color='b', alpha=0.6)
ax3.fill_betweenx(time[:-1], 0, synavoadi[5], 
                  where=(synavoadi[5]<0),color='r', alpha=0.6)
ax3.set_xlabel('Near=5deg',color='k', fontsize=12) 

ax4 = axs2[4]; ax4.plot(synavoadi[15],timeadi[:-1],'k')              # plot kolom-5
ax4.plot(synavo[15], time[:-1], 'k', lw=1.5, linestyle="--")
ax4.fill_betweenx(time[:-1], 0, synavoadi[15], 
                  where=(synavoadi[15]>0),color='b', alpha=0.6)
ax4.fill_betweenx(time[:-1], 0, synavoadi[15], 
                  where=(synavoadi[15]<0),color='r', alpha=0.6)
ax4.set_xlabel('Mid=15deg',color='k', fontsize=12) 

ax5 = axs2[5]; ax5.plot(synavoadi[35],timeadi[:-1],'k')                 # plot kolom-6
ax5.plot(synavo[35], time[:-1], 'k', lw=1.5, linestyle="--")
ax5.fill_betweenx(time[:-1], 0, synavoadi[35], 
                  where=(synavoadi[35]>0),color='b', alpha=0.6)
ax5.fill_betweenx(time[:-1], 0, synavoadi[35], 
                  where=(synavoadi[35]<0),color='r', alpha=0.6)
ax5.set_xlabel('Far=35deg',color='k', fontsize=12) 
top=thickness[0]+thickness[1]
base=thickness[0]+thickness[1]+thickness[2]
for aa in axs2:
    aa.xaxis.set_label_position('top')
    aa.axhline(y=top, linewidth=1, color='r')
    aa.axhline(y=base, linewidth=1, color='b')
for bb in [ax3, ax4, ax5]:
    bb.set_xlim(-0.25,0.25)

plt.show()

Pekerjaan di kelas, ubah beberapa parameter seperti porositas, saturasi, vsh, elastic logs, wavelet phase dsb

### HOMEWORK FOR FUN ...!!

Buat visualisasi partial stack dengan coupling effect: perubahan thickness layer-3 menjadi (h=10m) dan gas saturasi (sg=5% ) mixing fluid (brine+gas) secara uniform (Reuss)...best of luck!!
            
            rho_qz=2.65;  k_qz=37;  mu_qz=44    # mineral properties, quartz (i.e., sands)
            rho_sh=2.81;  k_sh=15;  mu_sh=5     # mineral properties, clay (i.e., shales)
            rho_b=1.09;   k_b=2.8               # fluid properties, brine
            rho_o=0.78;   k_o=0.94              # fluid properties, oil
            rho_g=0.25;   k_g=0.06              # fluid properties, gas
            
Laporkan juga berapa persen perubahan Vp, Vs, Rho jika diFRM dengan 5% saturasi gas (~non-commercial accumulation)

# <h1><center>-Akhir Program Latihan-9a-<center></h1>